In [69]:
import numpy as np

class CompressedMatrix:
    __array_priority__ = 1
    n, m = 0, 0 # rows count and columns count
    Matrix = [] # array of tuples (i, j, a_ij)
    Columns = [] 

    def __init__(self, n, m, matrix):
        self.n = n
        self.m = m
        self.Matrix = matrix
        self.Columns = [[] for i in range(self.m)]
        for i, j, a in self.Matrix:
            self.Columns[j].append([i, a])

    def __matmul__(self, other):
        if self.m != other.shape[0]:
            raise Exception('Exception')
        res = np.zeros(self.n)
        for i, j, a in self.Matrix:
            res[i] += a * other[j]
        return res

    def __rmatmul__(self, other):
        if self.n != other.shape[0]:
            raise Exception('Exception')
        res = np.zeros(self.m)
        for i, j, a in self.Matrix:
            res[j] += other[i] * a
        return res
    
    def Transposed(self):
        return CompressedMatrix(self.m, self.n, [[j, i, a] for i, j, a in self.Matrix])
      
    def __mul__(self, k):
        return CompressedMatrix(self.n, self.m, [[i, j, a * k] for i, j, a in self.Matrix])

    def multiplyRowByColumn(self, row, k):
        res = 0
        for [i, a] in self.Columns[k]:
            res += a * row[i]
        return res

In [70]:
A = CompressedMatrix(2, 3, [[0, 1, 1], [1, 2, 2]])
B = np.array([1, 2, 3])
C = np.array([2, 3])
print(A @ B)
print(C @ A)
print(A.Transposed().Matrix)
print((A * 2).Matrix)

[2. 6.]
[0. 2. 6.]
[[1, 0, 1], [2, 1, 2]]
[[0, 1, 2], [1, 2, 4]]


In [71]:
def ConjugateGradient(A, b):
    x = np.zeros(A.m)
    At = A.Transposed()
    b = At @ b
    v = (At @ (A @ x) - b)
    d = v
    v_norm = np.dot(v, v)
    
    for i in range(len(b)):
        Ad = At @ (A @ d)
        alpha = v_norm / (d @ Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v, v)

        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
    return x

In [72]:
import cvxpy as cp

def TrueSolution(A, b):
    x = cp.Variable(A.shape[1])
    cp.Problem(cp.Minimize(cp.sum_squares(A @ x - b)), []).solve()
    return x.value

n, m = 5, 5

for i in range(10):
    A = np.random.rand(n, m)
    b = np.random.rand(n)

    data = []
    for i in range(n):
        for j in range(m):
            data.append([i, j, A[i][j]])


    myRes = ConjugateGradient(CompressedMatrix(n, m, data), b)
    trueRes = TrueSolution(A, b)
    for i in range(m):
        assert(abs(myRes[i] - trueRes[i]) < 1e-5)